# Tutorial: What-if Scenarios

This notebook applies scenario transforms to base series inputs and visualizes outputs.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from tollama.core.scenarios import apply_scenario
from tollama.core.schemas import SeriesInput, WhatIfScenario

In [ ]:
base_series = [
    SeriesInput.model_validate({
        'id': 'store_001',
        'freq': 'D',
        'timestamps': ['2025-01-01','2025-01-02','2025-01-03','2025-01-04','2025-01-05'],
        'target': [120, 124, 126, 129, 132],
        'past_covariates': {'promo': [0, 1, 0, 1, 1]},
        'future_covariates': {'promo': [1, 1, 0]},
    })
]
base_series[0].model_dump(exclude_none=True)

In [ ]:
demand_up = WhatIfScenario.model_validate({
    'name': 'demand_up_15pct',
    'transforms': [{'operation': 'multiply', 'field': 'target', 'value': 1.15}],
})
promo_boost = WhatIfScenario.model_validate({
    'name': 'promo_future_boost',
    'transforms': [
        {'operation': 'replace', 'field': 'future_covariates', 'key': 'promo', 'value': 1}
    ],
})
scenario_outputs = {
    demand_up.name: apply_scenario(series=base_series, scenario=demand_up),
    promo_boost.name: apply_scenario(series=base_series, scenario=promo_boost),
}
scenario_outputs[demand_up.name][0].target

In [ ]:
base = pd.DataFrame({
    'step': list(range(1, len(base_series[0].target) + 1)),
    'baseline': base_series[0].target,
    'demand_up_15pct': scenario_outputs['demand_up_15pct'][0].target,
})
base

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(base['step'], base['baseline'], marker='o', label='baseline')
ax.plot(base['step'], base['demand_up_15pct'], marker='o', label='demand_up_15pct')
ax.set_title('Scenario-transformed target values')
ax.set_xlabel('step')
ax.set_ylabel('value')
ax.grid(alpha=0.2)
ax.legend()
plt.tight_layout()

In [ ]:
fig = go.Figure()
fig.add_scatter(
    x=base['step'],
    y=base['baseline'],
    mode='lines+markers',
    name='baseline',
)
fig.add_scatter(
    x=base['step'],
    y=base['demand_up_15pct'],
    mode='lines+markers',
    name='demand_up_15pct',
)
fig.update_layout(
    title='Scenario comparison',
    xaxis_title='step',
    yaxis_title='value',
)
fig

In [ ]:
bad_scenario = WhatIfScenario.model_validate({
    'name': 'invalid_missing_key',
    'transforms': [
        {'operation': 'add', 'field': 'future_covariates', 'key': 'not_present', 'value': 1}
    ],
})
try:
    apply_scenario(series=base_series, scenario=bad_scenario)
except ValueError as exc:
    print('Error handling example:', exc)